# Technical test for the data science internship @Hyperlex

In [18]:
import os
import pandas as pd

rootdir = 'dataset'
json_files = [file for _,_,files in os.walk(rootdir) for file in files if file.endswith('json')]      
      

[' STATUTS 2-OCR_OCR-pdfa.json',
 '161102 statuts à jour -OCR_OCR-pdfa.json',
 '17. Statuts post RK-OCR_OCR-pdfa.json',
 '2016 06 12 Statuts-OCR_OCR-pdfa.json',
 '2016 09 29 Statuts modifiés AssurCopro - copie-OCR_OCR.json',
 '2016 09 29 Statuts modifiés AssurCopro Sud-OCR_OCR.json',
 '2016 09 29 Statuts modifiés Axim Investment-OCR_OCR-pdfa.json',
 'A VERIFIER STATUTS-OCR_OCR-pdfa.json',
 'BizNet.io - Statuts constitutifs-OCR_OCR.json',
 'Copie de STATUTS GD RHONE ALPE SIGNES 25 10 2016 (conflit du 31-10-2016 à 11h39)-OCR_OCR.json',
 'HELENE BLANC - Statuts 160617-OCR_OCR.json',
 'LOGOS STATUTS-OCR_OCR-pdfa.json',
 'PMG - Statuts modifiés - 22.12.2016-OCR_OCR-pdfa.json',
 'POWER MAN - STATUTS-OCR_OCR-pdfa.json',
 'STATUT 2-OCR_OCR-pdfa.json',
 'STATUT DIMTI INSTRU_20160120123017-OCR_OCR.json',
 'STATUTS  2-OCR_OCR-pdfa.json',
 'STATUTS  3-OCR_OCR-pdfa.json',
 'STATUTS  4-OCR_OCR.json',
 'STATUTS (1) 2-OCR_OCR-pdfa.json',
 'STATUTS (1) 3-OCR_OCR.json',
 'STATUTS (1)-OCR_OCR-pdfa.json',

In [12]:

#One option is listing all files in a directory with os.listdir and then finding only those that end in '.json':

import os, json
import pandas as pd

path_to_json = 'dataset/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(json_files)  # for me this prints ['foo.json']

[]
